In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt                     
from numpy.fft import fft2, fftshift, ifft2

In [ ]:
def num(str_name):
  return str_name[65:-5]
v_num=np.vectorize(num)

In [ ]:
x_ind=v_num(glob.glob('/content/drive/MyDrive/INDIGO_MLR/DataAugmentation/preview/cat_0_*.jpeg'))

In [ ]:
path_base='/content/drive/MyDrive/INDIGO_MLR/DataAugmentation/preview/cat_0_{}.jpeg'

In [ ]:
def get_data(nodule_id):    
    ima = cv2.imread(path_base.format(nodule_id)).astype('float32')
    ima=(ima-np.min(ima))/(np.max(ima)-np.min(ima))
    return ima, 1

In [ ]:
def fourier_fase(image_F):
  Mod=np.abs(image_F)
  Mod=Mod*np.random.rand(np.shape(Mod)[0], np.shape(Mod)[1])
  fas=np.arctan(image_F.imag/image_F.real)
  pi_n=-np.logical_and(image_F.imag<0,image_F.real<0).astype('int')*np.pi
  pi_p=np.logical_and(image_F.imag>0,image_F.real<0).astype('int')*np.pi
  fas=fas+pi_n+pi_p
  image_F=(Mod*np.cos(fas)+Mod*np.sin(fas)*1j)
  return ifft2(image_F)

In [ ]:
def dataugmentation(ima):
  out=np.zeros(np.shape(ima))
  if np.shape(ima)[-1]==3:
    for i in range(3):
      F=fft2(ima[...,i])
      inv=fourier_fase(F)
      out[...,i]=np.abs(inv)
  else:
    F=fft2(ima)
    inv=fourier_fase(F)
    out=np.abs(inv)
  out=(out-np.min(out))/(np.max(out)-np.min(out)).astype('float32')
  return out

In [ ]:
def one_hot(vec):
  vec = vec-1
  shape = (vec.size, int(vec.max()+1))
  one = np.zeros(shape)
  rows = np.arange(vec.size)
  one[rows, vec.astype('int')] = 1
  return one

In [ ]:
from keras.utils import Sequence
class Sequence_data(Sequence):
    def __init__(self, data, batch_size, is_training):
        self.data = data
        self.batch_size = batch_size
        self.is_training = is_training

    def __len__(self):
        return int(np.ceil(len(self.data) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.data[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        X = []
        Y = []

        for path in batch_x:
          data_x, data_y = get_data(path)
          X.append(data_x)
          Y.append(data_y)
          if self.is_training:
            data_x=dataugmentation(data_x)
            X.append(data_x)
            Y.append(data_y)           
        X = np.array(X)
        Y = np.array(Y)

        return X, one_hot(Y)

In [ ]:
seq_train = Sequence_data(x_ind, 30, True)